<a href="https://colab.research.google.com/github/weasel-codes/covid-patient-recovery/blob/main/Covid19_Patient_Recovery_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relevant imports

In [67]:
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd

# Importing Dataset

In [73]:
dataset = pd.read_csv('PatientInfo.csv')
dataset = dataset[{'sex','age','infection_case','confirmed_date','released_date','deceased_date', 'state'}]
print("Shape of Dataset : ", dataset.shape)
print("No. of Nan values per column : ")
dataset.isnull().sum()

Shape of Dataset :  (5165, 7)
No. of Nan values per column : 


age               1380
sex               1122
state                0
confirmed_date       3
infection_case     919
deceased_date     5099
released_date     3578
dtype: int64

In [74]:
from datetime import date

print("Shape of dataset before removing Nan from any column : ", dataset.shape)

dataset = dataset.dropna(subset=['confirmed_date'], how='all')
print("Shape of dataset after removing Nan from confirmed_date column : ", dataset.shape)

dataset = dataset.dropna(subset=['deceased_date', 'released_date'], how='all')
print("Shape of dataset after removing Nan from deceased_date and released_date column : ", dataset.shape)
print("\nNo. of Nan values per column after removing released, deceased, confirmed date : ")
print(dataset.isnull().sum())

dataset = dataset.dropna(subset=['infection_case'], how='all')
print("\nShape of dataset after removing Nan from infection_case column : ", dataset.shape)

print("No. of Nan values per column after removing Nan values from infection_case : ")
print(dataset.isnull().sum())

Shape of dataset before removing Nan from any column :  (5165, 7)
Shape of dataset after removing Nan from confirmed_date column :  (5162, 7)
Shape of dataset after removing Nan from deceased_date and released_date column :  (1651, 7)

No. of Nan values per column after removing released, deceased, confirmed date : 
age                 11
sex                  1
state                0
confirmed_date       0
infection_case     611
deceased_date     1585
released_date       64
dtype: int64

Shape of dataset after removing Nan from infection_case column :  (1040, 7)
No. of Nan values per column after removing Nan values from infection_case : 
age                 11
sex                  1
state                0
confirmed_date       0
infection_case       0
deceased_date     1012
released_date       27
dtype: int64


In [75]:
dataset = dataset.iloc[:,:].values
dataset

array([['50s', 'male', 'released', ..., 'overseas inflow', nan,
        '2020-02-05'],
       ['30s', 'male', 'released', ..., 'overseas inflow', nan,
        '2020-03-02'],
       ['50s', 'male', 'released', ..., 'contact with patient', nan,
        '2020-02-19'],
       ...,
       ['30s', 'female', 'released', ..., 'Itaewon Clubs', nan,
        '2020-06-12'],
       ['30s', 'female', 'released', ..., 'overseas inflow', nan,
        '2020-06-13'],
       [nan, nan, 'released', ..., 'overseas inflow', nan, '2020-06-24']],
      dtype=object)